## Práctica 2. Procesamiento de lenguaje natural

Grupo 16: Adina Han y Diego Ambite

### Parte 2.  Recuperación de información

### Apartado a)
#### El conjunto está dividido de forma predeterminada en entrenamiento y prueba en porcentajes de 60 y 40, respectivamente (como se puede ver en el notebook de prueba). Usaremos únicamente la parte de entrenamiento como los mensajes a recuperar por nuestro buscador.

#### Vamos a utilizar una representación de la bolsa de palabras de countVectorizer con las siguientes opciones:
    - La bolsa de palabras tendrá en cuenta la frecuencia de las palabras en cada mensaje (binary=False)
    - Usa el diccionario que se encuentra en la siguiente URL y que ya usamos en el notebook de prueba. https://github.com/dwyl/english-words/blob/master/words.txt
    - Usa la lista de palabras vacías (parámetro stop_words) que proporciona sklearn para el inglés
    - Usando un rango de n-gramas de (1,1) (parámetro ngram_range).

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

In [ ]:
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

print("Training texts:", len(train_data.data))
print("Test texts:", len(test_data.data))

In [ ]:
df = pd.DataFrame(data=train_data.data)
df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
vectorizer = CountVectorizer(stop_words='english', binary=False, ngram_range=(1,1))
# Tomamos los textos del conjunto de entrenamiento y los transformamos en 
# una matriz de datos (palabras) según el diccionario estándar
train_vector_data=vectorizer.fit_transform(train_data.data)

In [ ]:
train_vector_data

In [ ]:
feature_names = vectorizer.get_feature_names()

print(len(feature_names))
print(feature_names)

In [ ]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [ ]:
#write_terms(feature_names, train_data.data, train_vector_data, 10)
write_terms(feature_names, None, train_vector_data, 10)

write_terms(feature_names, None, train_vector_data, 100)

In [ ]:
# Pasamos el fichero a una lista (una línea por item)
with open('words.txt') as f:
    dictionary = f.read().splitlines()

# El diccionario cargado lo pasamos en el parámetro vocabulary    
vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english')

In [ ]:
feature_names = vectorizer.get_feature_names()

print(len(feature_names))

In [ ]:
# Calculamos el valor TF-IDF 
tfidfer = TfidfTransformer()
train_preprocessed = tfidfer.fit_transform(train_vector_data) 

In [ ]:
train_preprocessed

In [ ]:
#write_terms(feature_names, train_data.data, train_vector_data, 10)
write_terms(feature_names, None, train_vector_data, 10)

write_terms(feature_names, None, train_vector_data, 100)

#### Para calcular la similitud entre dos mensajes usaremos la similitud del coseno (sklearn.metrics.pairwise.cosine_similarity) que es capaz de medir la similitud entre los elementos (es decir, entre las filas) de dos matrices de vectores de términos pudiendo ser estas matrices densas o dispersas.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
import random 
results=[]
for i in train_data.target_names:
    print("i", i)
    for j in range(3):
        print("j", j)
        print("len", len(train_data.data)-1)
        count=random.randint(0,len(train_data.data)-1)
        
        print("count", count)
        print("data", train_data.data[count])
        print("data-target name", train_data.target_names[count])

        while train_data.target_names[count] != i:
            count=random.randint(0,len(train_data.data)-1)
        results.append(train_data[count])
results

In [ ]:
for i in range(len(train_vector_data.data)):
    print("-------------------------------------------------")
    print(train_data.data[i])

#### Toma 3 mensaje del conjunto de prueba para cada clase (es decir, para cada tema). Vas a usar cada uno de dichos mensajes como consulta para recuperar los mensajes del conjunto de entrenamiento que más se parezcan a la consulta. Para ello sigue los siguientes pasos:
    1. Usa la distancia del coseno entre el mensaje de consulta y los mensajes de entrenamiento.
    2. Ordena los resultados de mayor a menor relevancia con la consulta.
    3. Calcula la precisión de la lista de resultados con nivel de exhaustividad 3 y 10.
        - La precisión a un nivel de exhaustividad X es el número de resultados que son relevantes (es decir, de la clase buscada) de entre los X primeros recuperados.
    4. Calcula los valores de precisión media (para cada nivel de exhaustividad) para cada clase del conjunto de datos.

#### Se valorará el uso de funciones y la claridad del código, así como sus comentarios. Contesta a lo siguiente:
    • ¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?
    • Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.
        o ¿Con qué clases se ha confundido más dicha consulta? o ¿A qué crees que se deben los malos resultados?
#### Debes usar la parte de entrenamiento para construir la bolsa de palabras con frecuencia y bolsa de palabras con TF/IDF.

### Apartado b)
#### Repite la secuencia de pasos descritos en el apartado a) pero ahora usa TF-IDF para ponderar el peso de los términos de la bolsa de palabras. Para usar TF-IDF primero debes transformar los textos usando countVectorizer con binary=False para obtener la frecuencia de palabras (exactamente igual que en el apartado anterior), y a continuación usar TfidfTransformer para modular dicha frecuencia según lo popular que sea cada término en el conjunto de mensajes de entrenamiento.
#### A continuación contesta a lo siguiente.
    • ¿Han cambiado los valores de precisión media para las clases del conjunto de datos? ¿Qué clases han mejorado? ¿Cuáles han empeorado?
    • Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.